In [1]:

path = "abfss://609090c6-5f30-44b0-9851-1e1db81bd2b9@onelake.dfs.fabric.microsoft.com/58b436c1-5447-4837-a300-9856faaa0806/Tables/nyc_tlc_data"

df = spark.read.format("delta").load(path)


StatementMeta(, d7ca36ac-3bf9-4081-8a02-b710f253dc4b, 3, Finished, Available, Finished)

#### Create Agg Table

In [2]:
from pyspark.sql.functions import *

date_df = df.withColumn('PickupDate', to_date('lpepPickupDatetime')) \
    .withColumn('DropOffDate', to_date('lpepDropoffDatetime'))

# date_df = date_df.filter(date_df['vendorID'] == '1')

# date_df = date_df.filter(date_df['PickupDate'] == '2019-06-21')

#agg
'''
passengerCount
tripDistance
fareAmount
tipAmount
tollsAmount
totalAmount
tripType
puYear
puMonth
'''


date_df = date_df.groupBy(['vendorID','tripType','PickupDate','DropOffDate','puYear','puMonth']) \
    .agg(sum("passengerCount").alias("passengerCount"), \
         sum("tripDistance").alias("tripDistance"), \
         sum("fareAmount").alias("fareAmount"), \
         sum("tollsAmount").alias("tollsAmount"), \
         sum("totalAmount").alias("totalAmount"), \
    )    

date_df_cols = date_df.columns

date_df = date_df.select(*[concat(lit(_col),lit(": "),col(_col)).alias(_col) for _col in date_df.columns])
date_df = date_df.withColumn('combined', concat_ws(",", *date_df_cols))

date_df = date_df.withColumn('role',lit("system"))

date_df.write.format("delta").mode("overwrite").saveAsTable("nyc_tlc_agg")
#content_list = date_df.toPandas().to_dict("records")


StatementMeta(, d7ca36ac-3bf9-4081-8a02-b710f253dc4b, 4, Finished, Available, Finished)

In [4]:
AZURE_OPENAI_API_KEY ="467edb37-1fc0-44e8-87d6-b45a674e52c8" 
AZURE_OPENAI_ENDPOINT= "https://mango-bush-0a9e12903.5.azurestaticapps.net/api/v1" 

StatementMeta(, ae060c76-9803-4f6e-b6a6-fd36e3f786f3, 6, Finished, Available, Finished)

In [ ]:
def get_message(content:list, question:str):
    MESSAGES = [{"role": "system", "content": "You are a helpful assistant."}]
    
    for item in content:
        tmp = {"role":"system","content":str(item)}
        MESSAGES.append(tmp)

    user_content = {"role": "user", "content": question}

    MESSAGES.append(user_content)

    return MESSAGES    

def get_content(input_parameter : str, input_value : str):
    content = []
    return content

def get_response(user_question: str):
    from openai import AzureOpenAI
    import json
        
    client = AzureOpenAI(
        api_key=AZURE_OPENAI_API_KEY,  
        api_version="2024-02-01",
        azure_endpoint = AZURE_OPENAI_ENDPOINT
        )
    MODEL_NAME = 'gpt-4-turbo-2024-04-09'#'gpt-35-turbo'

    content = get_content(input_parameter,input_value)

    MESSAGE = get_message(content)

    completion = client.chat.completions.create(
        model=MODEL_NAME,
        messages=MESSAGES,
        )

    


In [5]:
!pip install openai

StatementMeta(, ae060c76-9803-4f6e-b6a6-fd36e3f786f3, 7, Finished, Available, Finished)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.3/389.3 kB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 31.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 32.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.3/327.3 kB 91.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.9/434.9 kB 107.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 107.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 26.8 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.9.0
    Uninstalling typing_extensions-4.9.0:
      Successfully uninstalled typing_extensions-4.9.0


In [7]:
from openai import AzureOpenAI
import json
    
client = AzureOpenAI(
    api_key=AZURE_OPENAI_API_KEY,  
    api_version="2024-02-01",
    azure_endpoint = AZURE_OPENAI_ENDPOINT
    )
MODEL_NAME = 'gpt-4-turbo-2024-04-09'#'gpt-35-turbo'


MESSAGES = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "system", "content": 'vendorID: 1,tripType: 2,PickupDate: 2019-06-21,DropOffDate: 2019-06-21,puYear: 2019,puMonth: 6,passengerCount: 70,tripDistance: 276.0,fareAmount: 597.2,tollsAmount: 24.48,totalAmount: 654.9300000000001'},
    {"role": "system", "content": 'vendorID: 1,tripType: 1,PickupDate: 2019-06-21,DropOffDate: 2019-06-21,puYear: 2019,puMonth: 6,passengerCount: 3087,tripDistance: 7747.600000000007,fareAmount: 41109.80000000007,tollsAmount: 491.8700000000004,totalAmount: 47980.37000000139'},
    {"role": "user", "content": "How many types of trip Types are there"}
    # {
    #     "role": "assistant",
    #     "content": "The Los Angeles Dodgers won the World Series in 2020.",
    # },
    # {"role": "user", "content": "Where was it played?"},
]

completion = client.chat.completions.create(
    model=MODEL_NAME,
    messages=MESSAGES,
)

response = completion.choices[0].message
print(response.content)

StatementMeta(, ae060c76-9803-4f6e-b6a6-fd36e3f786f3, 9, Finished, Available, Finished)

There are two types of trip types mentioned in the data provided:

1. Trip Type 1
2. Trip Type 2
